In [1]:
from src.Slm import slm
import numpy as np
import matplotlib.pyplot as plt
import csv

# Loading parmaters
E = 200e9 # Elastic modulus (Pa)
D = 1 # Diameter mm
P = 7850 # Density (kg/m^3)
F = [0,-1,0,0,0,0,0,0]

# Simulation Parameters
a = .1 # A link set constant here
file_name = "./results/simulation/pareto_front.csv"

# Create new file with column headers
with open(file_name, 'w', newline='') as csvfile:
    slm_writer = csv.writer(csvfile, delimiter=' ',
            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    slm_writer.writerow(["Rr","Rf","Rh","Rw","A","k_max","k_min","Range","Weight","Height"])

step_size = a/10
n = 15

# System parameters for Class I
rw_lower,rw_upper = 0.2*a,4*a
rh_lower,rh_upper = -3*a,4*a
s_lower,s_upper = -a,0  
s_pad = 0.15*a # how far from singular point

# Rhombus parameter range
Rw_range = np.linspace(rw_lower,rw_upper,n)
Rh_range = np.concatenate((np.linspace(rh_lower,s_lower-s_pad,n),
            np.linspace(s_lower+s_pad,s_upper-s_pad,n),np.linspace(s_upper+s_pad,rh_upper,n)))

# Reflection Ratio range
Rr_range = np.linspace(1,4,n)
Rr_range_rf = np.linspace(2.5,4,n)
SLM = slm.mechanism(Rr=1,Rf=False,A=1,Rh=3,Rw=1)

# Collects and writes properties of mechanisms
def write_properties(Rr,Rf,A,Rh,Rw):
    try:
        SLM.update_links(Rr,Rf,A=A,Rh=Rh,Rw=Rw)
        k_max,k_min,range,weight,height = SLM.find_system_properties(F,E,D,P)
        with open(file_name, 'a', newline='') as csvfile:
            slm_writer = csv.writer(csvfile, delimiter=' ',
                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
            slm_writer.writerow([Rr,Rf,Rh,Rw,A,k_max,k_min,range,weight,height])
    except Exception as e: print("Error encountered:",e)

print(len(Rh_range),len(Rw_range),len(Rr_range),len(Rr_range_rf))

# Full Mechanism Sweep
for Rh in Rh_range:
    for Rw in Rw_range:
        Rf = False
        for Rr in Rr_range:
            if Rr*Rw < rw_upper: write_properties(Rr,Rf,a,Rh,Rw)
        Rf = True
        for Rr in Rr_range_rf:
            if Rr*Rw < rw_upper: write_properties(Rr,Rf,a,Rh,Rw)

45 15 15 15
